In [1]:
# default_exp sandbox
# hide
_FNAME='sandbox'

import unittest
from unittest import mock
from nbdev.export import notebook2script
import os
TESTCASE = unittest.TestCase()
_nbpath = os.path.join(_dh[0], _FNAME+'.ipynb')
notebook2script(_nbpath)

Converted sandbox.ipynb.


In [2]:
#export
from mranderson import NEO4J_VERSION
from mranderson.db import Query
import mranderson.db as db
import scylla

In [3]:
#export
def is_test_data(driver=None):
    query = Query(driver)
    query.add("MATCH (t:Totem {eid:'totem'}) return t.is_test_data")
    try:
        return query.single()
    except NoSearchResults:
        return False

def is_empty(driver=None):
    query = Query(driver)
    query.add("MATCH (n) RETURN n LIMIT 1")
    data = query.data()
    return len(data) == 0

def create_totem(driver=None):
    query = Query(driver)
    query.add("CREATE (totem: Totem {eid:'totem', is_test_data: true})")
    return query.create()

class NotTestDataset(Exception):
    pass


def start_fresh(driver=None):
    if is_empty(driver):
        return True
    elif is_test_data(driver) is True:
        query = Query(driver)
        query.add("MATCH (n) DETACH DELETE n")
        query.create()
        return True
    else:
        raise NotTestDataset('This dataset is not marked as "for testing" by a totem and will not be deleted')
        

In [4]:
#export
def run_neo4j(version=NEO4J_VERSION, run_on_set_ports=False, **kwargs):
    """Spin up a new neo4j in a docker instance if one is not already running.
    Set the ichordb default driver to point to this test database.
        """
    image = "neo4j:{}".format(version)
    if run_on_set_ports:
        ports = {7687:17687, 7474:17474, 7473:17473}
    else:
        ports = [7687, 7474, 7473]

    additional_kwargs = {'environment': {'NEO4J_AUTH': db.DEFAULT_AUTH}}
    additional_kwargs.update(kwargs)
    neo4j = scylla.run_container(image, ports=ports,
                          block_until_port_available=7474,
                          **additional_kwargs)
    neo4j = scylla.set_port_name(neo4j, 7687, 'bolt')
    neo4j = scylla.set_port_name(neo4j, 7474, 'browser')
    neo4j = scylla.set_port_name(neo4j, 7473, 'secure_browser')
    return neo4j

In [5]:
#export
def start_neo4j(*args, **kwargs):
    '''
    Convenience function to
    Runs a neo4j container and returns the container and the driver
    '''
    container = run_neo4j(*args, **kwargs)
    driver = db.make_driver(container=container)
    db.set_default_driver(driver)
    return container, driver

In [6]:
container_name = 'mranderson_unittest_sandbox'
container, driver = start_neo4j(container_name=container_name)

In [7]:
TESTCASE.assertTrue(start_fresh(driver=driver))
TESTCASE.assertEqual(create_totem(driver).nodes_created, 1)

In [8]:
container.stop()